In [56]:
import os
import sys
import shutil
import pandas as pd

When using this code you need to manually do the following things:
1. Manually remove the vtk and csv file of the corrupted mesh
2. Manually remove the dice value of the corrupted mesh from the end_dice.csv file
3. Manually remove the thickness results calculated for the corpted time frame both using the segmentation masks and the meshes
4. Manually delete the volumes for the corrupted time frame from the seg_volumes_cal.csv file in seg_analysis_data/meshes and seg_analysis_data/segmentation


In [57]:
# Remove corrupted time step from bp_dice_history.csv and myo_dice_history.csv


dataset = "SCD0004301"
time_step = 8

# Load CSV as DataFrame, then convert to matrix (NumPy array)
df_bp_dice = pd.read_csv(f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/bp_dice_history.csv")  # header=None if no column names
 
df_bp_dice_modi = df_bp_dice.drop(columns=[f'{time_step}'])
print(df_bp_dice_modi.head())

df_bp_dice_modi.to_csv(f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/bp_dice_history.csv", index=False)


   Unnamed: 0         0         1         2         3         4         5  \
0           0  0.038943  0.032484  0.027740  0.022147  0.013936  0.008536   
1           1  0.739524  0.752690  0.723575  0.687283  0.548906  0.479932   
2           2  0.752561  0.766744  0.756008  0.692573  0.587939  0.486589   
3           3  0.760625  0.782614  0.771401  0.713236  0.621113  0.493183   
4           4  0.762399  0.735344  0.754241  0.737052  0.634015  0.508335   

          6         7         9        10        11        12        13  \
0  0.009386  0.010064  0.013170  0.019365  0.026439  0.025564  0.030648   
1  0.420798  0.500722  0.578526  0.651013  0.707112  0.639269  0.739747   
2  0.446045  0.494823  0.619926  0.661906  0.713750  0.655106  0.742843   
3  0.483665  0.509025  0.568350  0.641140  0.717516  0.651172  0.768118   
4  0.491801  0.461088  0.606595  0.681182  0.733100  0.650773  0.766094   

         14        15        16        17        18  
0  0.029295  0.028159  0.028129 

In [58]:
# Load CSV as DataFrame, then convert to matrix (NumPy array)
df_myo_dice = pd.read_csv(f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/myo_dice_history.csv")  # header=None if no column names
 
df_myo_dice_modi = df_myo_dice.drop(columns=[f'{time_step}'])
print(df_myo_dice_modi.head())

df_myo_dice_modi.to_csv(f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/myo_dice_history.csv", index=False)

   Unnamed: 0         0         1         2         3         4         5  \
0           0  0.104644  0.106866  0.114044  0.123438  0.137948  0.147996   
1           1  0.621290  0.680885  0.713344  0.664866  0.647506  0.577770   
2           2  0.700927  0.689289  0.692487  0.654055  0.630015  0.645349   
3           3  0.685131  0.746536  0.729598  0.675799  0.627196  0.663178   
4           4  0.706597  0.677680  0.730368  0.680822  0.635672  0.665627   

          6         7         9        10        11        12        13  \
0  0.146459  0.143111  0.133236  0.126241  0.118148  0.095105  0.108103   
1  0.559268  0.613712  0.613414  0.638336  0.694896  0.602628  0.718084   
2  0.598872  0.625079  0.644851  0.686695  0.722704  0.627523  0.720424   
3  0.630510  0.652957  0.560351  0.661083  0.708967  0.644517  0.706232   
4  0.644617  0.543580  0.628030  0.685513  0.746100  0.610491  0.738256   

         14        15        16        17        18  
0  0.107629  0.106967  0.106046 

In [59]:
#recalculate the ED, ES and EF state and calculate and plot volumes from the meshes

from Python_Code.Utilis.analysis_utils import cal_bp_volume, plot_time_series, _calculate_cardiac_parameters

meshes_dir = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/meshes"
destination_dir = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/mesh_analysis_data"

os.makedirs(meshes_dir, exist_ok=True)

# Initialize tracking variables
volumes_data = []
max_bp_volume = float('-inf')
min_bp_volume = float('inf')
max_bp_file = None
min_bp_file = None

# Process all VTK files
for filename in os.listdir(meshes_dir):
    if not filename.lower().endswith('.vtk'):
        continue
        
    time_frame = int(filename.split('=')[1].split('.')[0])
    vtk_path = os.path.join(meshes_dir, filename)
    
    # Calculate volumes
    bp_volume, myo_volume = cal_bp_volume(vtk_path)
    bp_volume_ml = bp_volume * 1e-3
    myo_volume_ml = myo_volume * 1e-3
    
    volumes_data.append({
        'bp_volume': bp_volume_ml,
        'myo_volume': myo_volume_ml,
        'Time': time_frame
    })
    
    # Track extremes for ED/ES identification
    if bp_volume > max_bp_volume:
        max_bp_volume = bp_volume
        max_bp_file = filename
    if bp_volume < min_bp_volume:
        min_bp_volume = bp_volume
        min_bp_file = filename

# Create and save results
df_volumes = pd.DataFrame(volumes_data).sort_values('Time').reset_index(drop=True)

df_ed_es = pd.DataFrame([
    {'state': 'ED', 'volume': max_bp_volume * 1e-3, 'time_frame': max_bp_file},
    {'state': 'ES', 'volume': min_bp_volume * 1e-3, 'time_frame': min_bp_file}
])

# Copy ED/ES mesh files to output directory
os.makedirs(destination_dir, exist_ok=True)

shutil.copy2(
    os.path.join(meshes_dir, max_bp_file),
    os.path.join(destination_dir, 'ED_state.vtk')
)
shutil.copy2(
    os.path.join(meshes_dir, min_bp_file),
    os.path.join(destination_dir, 'ES_state.vtk')
)

# Save results
df_ed_es.to_csv(os.path.join(destination_dir, 'ED_ES_states.csv'), index=False)
df_volumes.to_csv(os.path.join(destination_dir, 'mesh_volumes_cal.csv'), index=False)

# Generate plots
"""Generate volume plots for mesh analysis."""
output_folder = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/mesh_analysis_data/plots"
os.makedirs(output_folder, exist_ok=True)

titles = ["Volume of Blood Pool [ml]", "Volume of Myocardium [ml]"]

for i, col in enumerate(['bp_volume', 'myo_volume']):
    plot_time_series(
        df_volumes,
        file_name=col,
        output_folder=output_folder,
        title=titles[i],
        y_value=col,
        y_label=titles[i]
    )

print(f"ED state: {max_bp_file} (Volume: {max_bp_volume * 1e-3:.2f} ml)")
print(f"ES state: {min_bp_file} (Volume: {min_bp_volume * 1e-3:.2f} ml)")

ED state: mesh_t=18.vtk (Volume: 100.66 ml)
ES state: mesh_t=7.vtk (Volume: 49.42 ml)


In [60]:
#calculate outputs from slicing the meshes and analysing them using Simpsons method

df_mesh_seg_vol = pd.read_csv(f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/seg_analysis_data/meshes/seg_volumes_cal.csv")

out_folder_plots = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/seg_analysis_data/meshes/plots"

out_folder_analysis = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/seg_analysis_data/meshes"


# Generate plots
for i, col in enumerate(df_mesh_seg_vol.columns):
    plot_time_series(
        df_mesh_seg_vol, 
        file_name=col,
        output_folder=out_folder_plots,
        title=titles[i],
        y_value=col,
        y_label=titles[i]
    )

filename = 'seg_volumes_cal.csv'
# Save measurements to CSV
df_mesh_seg_vol.to_csv(os.path.join(out_folder_analysis, filename), index=False)

# Calculate and save cardiac parameters
cardiac_params = _calculate_cardiac_parameters(df_mesh_seg_vol['bp_volume'])
param_df = pd.DataFrame([
    {'Parameter': 'EDV', 'Value': cardiac_params['EDV'], 
    'Time_step': cardiac_params['EDV_time']},
    {'Parameter': 'ESV', 'Value': cardiac_params['ESV'], 
    'Time_step': cardiac_params['ESV_time']},
    {'Parameter': 'EF', 'Value': cardiac_params['EF'], 'Time_step': None},
    {'Parameter': 'SV', 'Value': cardiac_params['SV'], 'Time_step': None}
])
param_df.to_csv(
    os.path.join(out_folder_analysis, 'ED_ES_state.csv'), 
    index=False
)

In [61]:
#calculate outputs from analysing the segmentation masks using Simpsons method

df_mesh_seg_vol = pd.read_csv(f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/seg_analysis_data/segmentation/seg_volumes_cal.csv")

out_folder_plots = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/seg_analysis_data/segmentation/plots"

out_folder_analysis = f"/data.lfpn/ibraun/Code/paper_volume_calculation/outputs_patient_data/final_results/{dataset}/seg_analysis_data/segmentation"


# Generate plots
for i, col in enumerate(df_mesh_seg_vol.columns):
    plot_time_series(
        df_mesh_seg_vol, 
        file_name=col,
        output_folder=out_folder_plots,
        title=titles[i],
        y_value=col,
        y_label=titles[i]
    )

filename = 'seg_volumes_cal.csv'
# Save measurements to CSV
df_mesh_seg_vol.to_csv(os.path.join(out_folder_analysis, filename), index=False)


# Calculate and save cardiac parameters
cardiac_params = _calculate_cardiac_parameters(df_mesh_seg_vol['bp_volume'])
param_df = pd.DataFrame([
    {'Parameter': 'EDV', 'Value': cardiac_params['EDV'], 
    'Time_step': cardiac_params['EDV_time']},
    {'Parameter': 'ESV', 'Value': cardiac_params['ESV'], 
    'Time_step': cardiac_params['ESV_time']},
    {'Parameter': 'EF', 'Value': cardiac_params['EF'], 'Time_step': None},
    {'Parameter': 'SV', 'Value': cardiac_params['SV'], 'Time_step': None}
])
param_df.to_csv(
    os.path.join(out_folder_analysis, 'ED_ES_state.csv'), 
    index=False)